# Author Indentification using Text Snippets

Team Members:<br>
MT19019 - Suryank Tiwari <br>
MT19070 - Prateek Agarwal

Guide: Dr. Tanmoy Chakraborty

## Library Imports and File Loading

In [ ]:
#@title
!pip install contractions
!pip install textstat

In [ ]:
import nltk
import joblib
import string
import scipy
import numpy as np
import contractions
import pandas as pd
import xgboost as xgb
from textstat.textstat import *
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn import metrics, model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('treebank')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

!ls "/content/drive/My Drive/author_files"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CountVectorizer_DF.csv	test_unique.xlsx   train_unique.xlsx
df_features.pkl		test.xlsx	   train.xlsx
test_common.xlsx	train_common.xlsx  X_train.csv


In [ ]:
path_train = '/content/drive/My Drive/author_files/train.xlsx'
path_test = '/content/drive/My Drive/author_files/test.xlsx'

In [ ]:
train_df = pd.read_excel(path_train)
test_df = pd.read_excel(path_test)

train_df = train_df.replace('\n','', regex=True)
test_df = test_df.replace('\n','', regex=True)

# train_df = pd.read_excel('train_common.xlsx')
# test_df = pd.read_excel('test_common.xlsx')

In [ ]:
train_df

,id,text,author
0,24781,That is the most unbecoming thing there is,WO
1,20561,I call them forth to credit her,SW
2,62359,"She sat up and said as she had determined, ""My...",WV
3,28093,] But your career will have to be entirely dom...,WO
4,9022,Kent banish'd thus? and France in choler parte...,SW
...,...,...,...
36313,18837,"I'll make a shaft or a bolt on 't; 'slid, 'tis...",SW
36314,39617,""" Dorian Gray shook his head",WO
36315,9074,"My cue is villainous melancholy, with a sigh ...",SW
36316,5580,Enter Fortinbras with his Army over the stage,SW


In [ ]:
test_df

,id,text,author
0,37012,"Somehow, now that it has happened actually, a...",WO
1,7232,"As I walke thither, Ile tell ye more Both",SW
2,59845,"""They want to shut up,"" he said",WV
3,14685,Then 'banishment' Is death misterm'd,SW
4,17569,"With all my heart, so thou canst get a wife",SW
...,...,...,...
9075,48208,"Clacton, don't you think we should circulariz...",WV
9076,26665,"There is nothing but love, and I love her",WO
9077,54429,The great gas chandeliers were alight in the ...,WV
9078,56446,"Dalloway carried a despatch box, and his wife...",WV


In [ ]:
X_train = train_df.drop(['id', 'author'], axis = 1)
y_train = train_df['author']
X_test = test_df.drop(['id', 'author'], axis = 1)
y_test = test_df['author']

train_length=X_train.shape[0]

## Preprocessing methods
1. Tokenize
2. Lemmatize
3. Stopword Removal
4. Contraction expanding
5. Punctuation Removal
6. Lowercase conversion

In [ ]:
def preprocess(df):
    tokenized_list = []
    lemmatizer = WordNetLemmatizer() 
    stopwords_set = set(stopwords.words('english'))
    for index, row in df.iterrows():
        tokenized_sentence = nltk.word_tokenize(row['text'])
        lemmatized_sentence = []
        temp_dict = {}
        expanded_words = []
        for word in tokenized_sentence:
            expanded_words.append(contractions.fix(word))
        for word in expanded_words:
            lemmatized_sentence.append(lemmatizer.lemmatize(word))
        
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in lemmatized_sentence]
        stopword_removed_list = [word for word in stripped if not word in stopwords_set]
        stopword_removed_list = [word.lower() for word in stopword_removed_list if word]
        temp_dict['text'] = stopword_removed_list
        tokenized_list.append(temp_dict)

    df = pd.DataFrame(tokenized_list)
    return df

In [ ]:
frames = [X_train, X_test]
X = pd.concat(frames)
X

,text
0,That is the most unbecoming thing there is
1,I call them forth to credit her
2,"She sat up and said as she had determined, ""My..."
3,] But your career will have to be entirely dom...
4,Kent banish'd thus? and France in choler parte...
...,...
9075,"Clacton, don't you think we should circulariz..."
9076,"There is nothing but love, and I love her"
9077,The great gas chandeliers were alight in the ...
9078,"Dalloway carried a despatch box, and his wife..."


In [ ]:
processed_df = preprocess(X)
text = 'text'
processed_df

,text
0,"[that, unbecoming, thing]"
1,"[i, call, forth, credit]"
2,"[she, sat, said, determined, my, head, ache, i..."
3,"[but, career, entirely, domestic]"
4,"[kent, banish, would, thus, france, choler, p..."
...,...
45393,"[clacton, nt, think, circularize, province, pa..."
45394,"[there, nothing, love, i, love]"
45395,"[the, great, gas, chandelier, alight, office, ..."
45396,"[dalloway, carried, despatch, box, wife, dress..."


## Feature Extraction

In [ ]:
def extract_features(df, raw_df):

    # BASELINE FEATURES
    syllables=[]
    num_chars=[]
    num_terms=[] 
    average_syllables=[]
    num_unique_terms=[]
    avg_word_length = []

    punctuations = []
    stopw_count = []

    # VADER features
    vader_negatives=[]
    vader_neutrals=[]
    vader_positives=[]
    vader_compound_sentiments=[]

    word_counts = dict()
    word_lengths = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen']
    for length in word_lengths:
      word_counts[length]=[]
    sid = SentimentIntensityAnalyzer()
    count = lambda l1,l2: sum([1 for x in l1 if x in l2])

    for i in range(df.shape[0]):
        sentence = df[text].iloc[i]

        syllable_count=textstat.syllable_count(' '.join(sentence))
        character_count=sum(len(word) for word in sentence)
        term_count=len(sentence)
        avg_syllabs=round(float((syllable_count+0.001))/float(term_count+0.001),4)
        mean_word_length = np.mean([len(word) for word in sentence])

        syllables.append(syllable_count)
        num_chars.append(character_count)
        num_terms.append(term_count)
        average_syllables.append(avg_syllabs)
        num_unique_terms.append(len(set(sentence)))
        avg_word_length.append(mean_word_length)

        raw_sentence = raw_df['text'].iloc[i]
        punctuation_count = count(raw_sentence,set(string.punctuation))
        tokenized_sentence = nltk.word_tokenize(raw_sentence)
        stopwords_set = set(stopwords.words('english'))
        stopwords_sentence = [w for w in tokenized_sentence if w in stopwords_set]
        stopword_count = len(stopwords_sentence)

        punctuations.append(punctuation_count)
        stopw_count.append(stopword_count)

        # VADER
        vader_score = sid.polarity_scores(' '.join(sentence))
        vader_negatives.append(vader_score['neg'])
        vader_neutrals.append(vader_score['neu'])
        vader_positives.append(vader_score['pos'])
        vader_compound_sentiments.append(vader_score['compound'])

        word_length_counts = [0]*len(word_lengths)
        for length, label in enumerate(word_lengths):
          for word in sentence:
            try:
              word_length_counts[len(word)-1]+=1
            except:
              pass
        for length, label in enumerate(word_lengths):
          word_counts[label].append(word_length_counts[length])
          
    
    features = {'text_df':df[text], #'class':df['class'],
            'vader_negative':vader_negatives,
            'vader_neutral':vader_neutrals,
            'vader_positive':vader_positives,
            'syllables':syllables,
            'num_chars':num_chars,
            'num_terms':num_terms,
            'average_syllables':average_syllables,
            'num_unique_terms':num_unique_terms,
            'punctuation_count':punctuations,
            'stopword_count':stopw_count,
            'average_word_length':avg_word_length
    }
    for length in word_lengths:
      features['word_length_'+length]=word_counts[length] 

    feature_df = pd.DataFrame.from_dict(features)
    return feature_df

In [ ]:
df = extract_features(processed_df, X)
general_df = df.copy()
general_df = general_df.drop(columns=['text_df'], axis=1)

In [ ]:
df

,text_df,vader_negative,vader_neutral,vader_positive,syllables,num_chars,num_terms,average_syllables,num_unique_terms,punctuation_count,stopword_count,average_word_length,word_length_one,word_length_two,word_length_three,word_length_four,word_length_five,word_length_six,word_length_seven,word_length_eight,word_length_nine,word_length_ten,word_length_eleven,word_length_twelve,word_length_thirteen,word_length_fourteen,word_length_fifteen,word_length_sixteen
0,"[that, unbecoming, thing]",0.000,1.000,0.000,6,19,3,1.9997,3,0,5,6.333333,0,0,0,16,16,0,0,0,0,16,0,0,0,0,0,0
1,"[i, call, forth, credit]",0.000,0.435,0.565,5,16,4,1.2499,4,0,3,4.000000,16,0,0,16,16,16,0,0,0,0,0,0,0,0,0,0
2,"[she, sat, said, determined, my, head, ache, i...",0.200,0.615,0.185,14,45,11,1.2727,11,2,7,4.090909,16,32,32,48,16,0,16,0,0,16,0,0,0,0,0,0
3,"[but, career, entirely, domestic]",0.000,1.000,0.000,9,25,4,2.2497,4,1,5,6.250000,0,0,16,0,0,16,0,32,0,0,0,0,0,0,0,0
4,"[kent, banish, would, thus, france, choler, p...",0.091,0.815,0.094,41,144,28,1.4643,26,15,10,5.142857,0,16,48,160,0,160,16,32,0,16,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45393,"[clacton, nt, think, circularize, province, pa...",0.000,0.781,0.219,23,86,16,1.4375,16,10,15,5.375000,0,32,16,64,48,16,32,16,16,0,16,0,0,0,0,0
45394,"[there, nothing, love, i, love]",0.771,0.229,0.000,6,21,5,1.2000,4,1,4,4.200000,16,0,0,32,16,0,16,0,0,0,0,0,0,0,0,0
45395,"[the, great, gas, chandelier, alight, office, ...",0.000,0.594,0.406,11,39,7,1.5713,7,0,3,5.571429,0,0,32,0,16,48,0,0,0,16,0,0,0,0,0,0
45396,"[dalloway, carried, despatch, box, wife, dress...",0.000,0.704,0.296,23,82,12,1.9166,12,2,7,6.833333,0,0,32,16,0,32,32,48,0,16,0,16,0,0,0,0


### Compute TF-IDF features

In [ ]:
def get_tf_idf_features():
  vectorizer = TfidfVectorizer(tokenizer=None, preprocessor=None, ngram_range=(1, 3), use_idf=True, smooth_idf=False, norm=None, decode_error='replace', max_features=7000, min_df=5, max_df=0.75)
  sentences=[]
  for sentence in processed_df[text]:
      sentences.append(' '.join(sentence))
  tf_idf_features = vectorizer.fit_transform(sentences).toarray()
  # tf_idf_feature_vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
  tf_idf_features = pd.DataFrame(tf_idf_features)
  return tf_idf_features
tf_idf_features = get_tf_idf_features()

### Compute POS Tag Features

In [ ]:
def get_pos_tag_features():
  pos_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
  sentence_pos_tags = []
  for sentence in processed_df[text]:
      tags = nltk.pos_tag([word for word in sentence if word])
      tags = [tag[1] for tag in tags]
      sentence_pos_tags.append(" ".join(tags))
  pos_tag_features = pos_vectorizer.fit_transform(pd.Series(sentence_pos_tags)).toarray()
  # pos_tag_feature_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names())}
  pos_tag_features = pd.DataFrame(pos_tag_features)
  return pos_tag_features
pos_tag_features = get_pos_tag_features()

### Reducing sparsity using SVD

In [ ]:
def SVD_Reduce(features, components=20):
  svd = TruncatedSVD(n_components=components, algorithm='arpack')
  svd.fit(features)
  train = pd.DataFrame(svd.transform(features.iloc[:train_length]))
  test = pd.DataFrame(svd.transform(features.iloc[train_length:]))
  return train, test

In [ ]:
components=20
tf_idf_svd_train, tf_idf_svd_test = SVD_Reduce(tf_idf_features, components)
pos_svd_train, pos_svd_test = SVD_Reduce(pos_tag_features, components)

In [ ]:
train_df = df.iloc[:train_length].reset_index(drop=True)
test_df = df.iloc[train_length:].reset_index(drop=True)
train_df = pd.concat([train_df, tf_idf_svd_train, pos_svd_train], axis=1)
test_df = pd.concat([test_df, tf_idf_svd_test, pos_svd_test], axis=1)
del tf_idf_svd_train, tf_idf_svd_test, pos_tag_features, pos_svd_train, pos_svd_test

for i in range(train_df.shape[0]):
  train_df.at[i, 'text_df']=' '.join(train_df.iloc[i]['text_df'])
for i in range(test_df.shape[0]):
  test_df.at[i, 'text_df']=' '.join(test_df.iloc[i]['text_df'])

### Count Vectorizer

In [ ]:
tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit(train_df['text_df'].values.tolist() + test_df['text_df'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text_df'])
test_tfidf = tfidf_vec.transform(test_df['text_df'])

author_mapping_dict = {'WO':0, 'SW':1, 'WV':2}
train_df['author']=y_train
train_y = train_df['author'].map(author_mapping_dict)
train_df=train_df.drop(columns=['author','text_df'],axis=1)

train_df

,vader_negative,vader_neutral,vader_positive,syllables,num_chars,num_terms,average_syllables,num_unique_terms,punctuation_count,stopword_count,average_word_length,word_length_one,word_length_two,word_length_three,word_length_four,word_length_five,word_length_six,word_length_seven,word_length_eight,word_length_nine,word_length_ten,word_length_eleven,word_length_twelve,word_length_thirteen,word_length_fourteen,word_length_fifteen,word_length_sixteen,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000,1.000,0.000,6,19,3,1.9997,3,0,5,6.333333,0,0,0,16,16,0,0,0,0,16,0,0,0,0,0,0,0.000707,0.000442,0.023141,-0.566808,0.549840,0.082383,-0.131081,0.032867,0.291781,0.245527,-0.098881,-0.002555,-0.408831,0.052773,-0.199756,-0.195080,-0.122849,0.342964,-0.064780,0.020601,0.620706,-0.225629,-0.210844,-0.259404,-0.123853,-0.101895,0.006527,-0.033000,0.025051,-0.035813,0.007536,-0.030467,0.012579,0.094503,-0.032075,0.066841,0.113300,-0.064844,0.068328,-0.020955
1,0.000,0.435,0.565,5,16,4,1.2499,4,0,3,4.000000,16,0,0,16,16,16,0,0,0,0,0,0,0,0,0,0,0.000367,0.000154,0.010973,-0.241317,0.219419,0.029182,0.059067,-0.004256,0.156555,-0.007885,-0.036370,0.003937,-0.005246,-0.008312,-0.001554,0.108036,-0.038504,0.011194,0.000224,-0.006365,0.428294,0.298966,-0.100712,0.000599,0.563159,-0.136918,-0.072439,-0.061530,-0.016456,-0.196309,0.276506,-0.006948,0.198750,-0.155198,-0.115910,0.032583,0.009668,0.014201,0.056489,0.028148
2,0.200,0.615,0.185,14,45,11,1.2727,11,2,7,4.090909,16,32,32,48,16,0,16,0,0,16,0,0,0,0,0,0,0.001272,0.001088,0.043779,-1.190319,1.264243,0.226966,-0.803084,0.139590,0.300991,-0.028450,-0.072654,-0.150329,-1.361961,0.055456,0.274908,1.537116,-0.265684,1.501772,-0.249010,1.307095,0.227100,0.027201,-0.000658,0.231510,-0.023437,-0.037186,0.169414,0.062530,0.022972,0.115253,0.018969,0.068157,-0.013584,-0.024604,-0.073492,0.031428,-0.061983,-0.003777,0.020258,0.030420
3,0.000,1.000,0.000,9,25,4,2.2497,4,1,5,6.250000,0,0,16,0,0,16,0,32,0,0,0,0,0,0,0,0,0.000265,0.000137,0.010713,-0.327593,0.368712,0.070321,-0.145550,0.037461,-0.013940,0.034405,-0.050125,-0.005974,-0.175899,0.011014,-0.164594,-0.009401,0.099923,0.366599,-0.034715,0.100338,0.185836,-0.097235,-0.099130,-0.000010,0.022680,0.286402,-0.006658,-0.088467,0.019579,-0.053793,-0.005452,-0.071704,0.011655,-0.025626,0.050633,-0.023824,0.250260,0.007660,-0.081891,-0.170033
4,0.091,0.815,0.094,41,144,28,1.4643,26,15,10,5.142857,0,16,48,160,0,160,16,32,0,16,0,0,0,0,0,0,0.017277,0.016906,0.227630,-4.428512,4.171699,0.590431,-1.144605,0.074594,7.302168,0.122156,-1.405450,1.224090,4.981772,-0.312475,-0.748181,-0.640914,-0.922442,-1.097538,-0.004776,0.020220,0.357608,-0.019880,-0.189513,0.181273,-0.099402,0.074957,-0.057862,-0.011200,0.015343,-0.013379,0.029758,0.101171,0.003874,0.054249,-0.025220,0.009352,0.025675,-0.027395,0.023420,-0.117704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36313,0.000,1.000,0.000,10,32,7,1.4285,7,6,6,4.571429,16,0,0,48,32,0,0,0,16,0,0,0,0,0,0,0,0.000424,0.000268,0.015636,-0.419128,0.448206,0.079633,0.091018,0.002790,0.373623,0.031138,-0.083056,-0.025677,-0.428119,0.011255,0.077679,0.304633,-0.025640,0.226277,0.010572,-0.047725,0.366891,0.009625,-0.074274,0.524889,-0.133974,-0.125590,-0.096464,0.100046,-0.090561,-0.104583,0.020800,-0.053895,-0.022339,0.042111,-0.011298,-0.006722,-0.164429,0.073278,0.022949,-0.166795
36314,0.318,0.682,0.000,6,19,4,1.4999,4,1,1,4.750000,0,0,0,32,16,16,0,0,0,0,0,0,0,0,0,0,0.000396,0.000288,0.011973,-0.276164,0.248584,0.032444,-0.376764,0.049643,-0.115641,-0.030327,-0.005019,-0.007927,-0.193329,0.036647,-0.005841,0.121815,0.045984,0.043413,-0.130442,0.662416,0.626143,-0.129217,-0.181433,-0.327470,-0.196225,-0.107816,0.092071,-0.024580,-0.132884,-0.006249,

In [ ]:
test_df=test_df.drop(columns=['text_df'],axis=1)
test_df

,vader_negative,vader_neutral,vader_positive,syllables,num_chars,num_terms,average_syllables,num_unique_terms,punctuation_count,stopword_count,average_word_length,word_length_one,word_length_two,word_length_three,word_length_four,word_length_five,word_length_six,word_length_seven,word_length_eight,word_length_nine,word_length_ten,word_length_eleven,word_length_twelve,word_length_thirteen,word_length_fourteen,word_length_fifteen,word_length_sixteen,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000,0.496,0.504,15,46,8,1.8749,8,3,10,5.750000,0,16,16,16,16,0,16,32,16,0,0,0,0,0,0,0,0.000370,0.000436,0.014450,-0.438787,0.480601,0.086647,-0.508157,0.084576,-0.155930,0.025161,-0.025165,-0.075509,-0.590471,0.044193,-0.326069,-0.076019,0.134408,-0.062572,-0.078637,0.398580,0.208724,-0.172802,-0.003071,0.038611,0.041286,0.288931,-0.002673,-0.096614,0.012177,-0.040847,0.020345,-0.048749,-0.015314,0.025498,-0.041912,0.093000,-0.054166,0.190914,0.023575,0.093813
1,0.000,1.000,0.000,9,28,8,1.1250,8,1,1,3.500000,16,32,16,32,16,0,16,0,0,0,0,0,0,0,0,0,0.000692,0.000482,0.021995,-0.505717,0.464247,0.064975,-0.078450,0.016025,0.391851,0.057083,-0.120444,0.055115,-0.323269,0.042409,0.024929,0.140099,-0.080188,0.357810,-0.041490,-0.035455,0.174098,-0.100049,-0.133032,-0.058454,0.083383,0.136441,-0.041637,-0.081145,-0.012734,0.055482,-0.043360,-0.067064,0.043884,0.032008,0.024112,0.199551,-0.058674,-0.051838,-0.082231,-0.077273
2,0.000,0.698,0.302,4,16,4,1.0000,4,3,3,4.000000,0,0,0,64,0,0,0,0,0,0,0,0,0,0,0,0,0.013338,-0.016295,0.025828,-0.384497,0.397294,0.046859,-0.495915,0.082160,-0.135226,-0.107877,-0.003213,-0.151260,-0.781270,0.029367,-0.673851,0.591721,-0.277524,0.863229,-0.273501,0.744856,0.064136,-0.060606,-0.024050,0.040057,0.175458,-0.038162,0.109358,0.060350,0.014146,0.138161,-0.083221,-0.003316,-0.110305,-0.006860,0.012827,0.012144,-0.038097,0.052415,0.003320,0.016502
3,0.438,0.562,0.000,10,34,6,1.6666,6,3,0,5.666667,0,16,0,16,16,16,16,0,0,16,0,0,0,0,0,0,0.001172,0.000763,0.041634,-1.054273,1.065196,0.183050,-0.260523,0.043960,2.081551,0.003523,-0.551596,0.451151,1.780097,-0.160543,-0.648469,-0.531742,0.114549,-0.165219,0.028463,0.006545,0.313805,0.176321,0.062639,0.012372,-0.015500,0.125912,0.032090,-0.009839,0.007630,0.031254,0.005241,0.002019,0.001682,-0.046721,0.093002,0.035593,0.027276,0.161954,0.208780,0.086704
4,0.000,1.000,0.000,6,25,6,1.0000,6,1,4,4.166667,0,0,16,48,32,0,0,0,0,0,0,0,0,0,0,0,0.001281,0.000942,0.046373,-1.356121,1.531484,0.289579,2.313435,-0.199077,-0.664281,-1.332385,-0.054957,-0.008546,0.299467,-0.113024,-1.134582,1.214000,-1.352813,-0.418489,0.063720,-0.141393,0.573196,0.458432,-0.011880,-0.022219,0.446876,-0.093229,0.000815,-0.064064,-0.061957,-0.163212,0.217939,-0.013311,0.185216,-0.082618,-0.093707,-0.000281,-0.012190,-0.012973,0.016451,0.037476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9075,0.000,0.781,0.219,23,86,16,1.4375,16,10,15,5.375000,0,32,16,64,48,16,32,16,16,0,16,0,0,0,0,0,0.009148,-0.007758,0.068512,-1.597507,1.472201,0.192122,-1.052693,0.167860,-0.119847,1.513564,-0.031441,-0.208042,-1.854059,-0.026439,-2.902875,1.527576,0.054889,2.647077,-0.518531,2.021853,0.347283,0.099978,-0.158935,0.143641,-0.049614,-0.094342,0.072543,0.021135,0.061993,0.030086,-0.006498,-0.014875,-0.054764,0.086428,0.073303,-0.039469,-0.081193,0.046002,-0.007257,-0.040574
9076,0.771,0.229,0.000,6,21,5,1.2000,4,1,4,4.200000,16,0,0,32,16,0,16,0,0,0,0,0,0,0,0,0,0.001209,0.000712,0.043868,-1.179330,1.224134,0.204248,0.629474,-0.015666,-0.301511,0.779851,-0.135853,0.051310,-0.638262,0.040792,0.102996,-0.373720,1.866658,3.161077,0.093588,-2.498963,0.487989,0.329613,0.012569,0.014123,0.248235,0.156544,0.044787,-0.067982,-0.048930,-0.081507,0.096362,

In [ ]:
test = test_df
train = train_df

In [ ]:
def get_final_df():
  global df, tf_idf_features
  df = pd.concat([df,tf_idf_features], axis=1)
  del tf_idf_features
  return df.iloc[:train_length], df.iloc[train_length:].reset_index(drop=True)

In [ ]:
def get_data():
  X_train, X_test = get_final_df()
  train_text = X_train['text_df']
  test_text = X_test['text_df']
  X_train = X_train.drop(columns=['text_df'], axis=1).reset_index(drop=True)
  X_test = X_test.drop(columns=['text_df'], axis=1).reset_index(drop=True)
  return X_train, X_test, train_text, test_text

X_train, X_test, train_text, test_text = get_data()

In [ ]:
# train.to_csv('/content/drive/My Drive/author_files/CountVectorizer_DF.csv',index=False)
# X_train.to_csv('/content/drive/My Drive/author_files/X_train.csv',index=False)

### Outlier Removal using Isolation Forest

In [ ]:
# def remove_outliers(x_data):
#     clf = IsolationForest(random_state = 0)
#     labels = clf.fit_predict(x_data)
#     print("Number of outliers using Isolation Forest: ",list(labels).count(-1))
#     rows_tar1 = []
#     for i in range(0,len(labels)):
#         if labels[i] == -1:   #outliers
#             rows_tar1.append(i)
#         x_data_new = x_data[~x_data.index.isin(rows_tar1)]
#     return x_data_new 

# def outlier(train): 
#     # Train has X_Train & y_train merged into a dataframe
#     dfLHP = train[train['author'] == 2 ]
#     dfLHP = dfLHP.reset_index()
#     dfWO = train[train['author'] == 1 ]
#     dfWO = dfWO.reset_index()
#     dfPEA = train[train['author'] == 0 ]
#     dfPEA = dfPEA.reset_index()

#     dfLHP = remove_outliers(dfLHP) 
#     dfWO = remove_outliers(dfWO) 
#     dfPEA = remove_outliers(dfPEA) 
#     #train = remove_outliers(train)
#     return pd.concat([dfLHP, dfWO, dfPEA], axis=0, sort=False)

In [ ]:
# y_train = y_train.replace("LHP",2)
# y_train = y_train.replace("WO",1)
# y_train = y_train.replace("PEA",0)
# train = pd.concat([X_train, y_train], axis=1, sort=False)

# train = outlier(train)
# y_train = train['author']
# y_train = y_train.replace({2:"LHP", 1:"WO", 0:"PEA"})
# X_train = train.drop(columns=['author'], axis=1)

## Classification

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

### Multinomial Naive Bayes

In [ ]:
model = MultinomialNB()
model.fit(train_tfidf, y_train)
y_pred = model.predict(test_tfidf)
pred_test_y = model.predict_proba(test_tfidf)
report = classification_report(y_test, y_pred)
print(report)
print('Log Loss:', metrics.log_loss(y_test, pred_test_y))

              precision    recall  f1-score   support

          SW       0.89      0.92      0.90      3144
          WO       0.82      0.69      0.75      2786
          WV       0.78      0.87      0.82      3150

    accuracy                           0.83      9080
   macro avg       0.83      0.83      0.83      9080
weighted avg       0.83      0.83      0.83      9080

Log Loss: 0.4298314872970045


### Logistic Regression

In [ ]:
pipe = Pipeline(
        [('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01, solver='liblinear'))),
        ('model', LogisticRegression(class_weight='balanced',penalty='l1', solver='liblinear'))])


In [ ]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
param_grid = [{}]
grid_search = GridSearchCV(pipe, param_grid, cv=StratifiedKFold(n_splits=5, random_state=42).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ................................................. , total=  32.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   32.0s remaining:    0.0s


[CV] ................................................. , total=  30.7s
[CV]  ................................................................
[CV] ................................................. , total=  18.9s
[CV]  ................................................................
[CV] ................................................. , total=  29.9s
[CV]  ................................................................
[CV] ................................................. , total=  31.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.4min finished


In [ ]:
y_preds_logistic = model.predict(X_test)
y_preds_logistic_proba = model.predict_proba(X_test)

In [ ]:
def convert_labels(test, preds):
  test=np.array(test)
  preds=np.array(preds)
  labels=np.unique(test)
  for itr, label in enumerate(labels):
    test[test==label]=itr
    preds[preds==label]=itr
  return test, preds

In [ ]:
report = classification_report( y_test, y_preds_logistic )
print(report)
test, preds = convert_labels(y_test, y_preds_logistic)

# Added to remove the error of ValueError: Unknown label type
test = test.astype('int')
print('Log Loss:', metrics.log_loss(test, y_preds_logistic_proba))

              precision    recall  f1-score   support

          SW       0.84      0.87      0.85      3144
          WO       0.71      0.72      0.71      2786
          WV       0.80      0.76      0.77      3150

    accuracy                           0.78      9080
   macro avg       0.78      0.78      0.78      9080
weighted avg       0.78      0.78      0.78      9080

Log Loss: 0.5023842908786998


### Passive Aggressive Classifier

In [ ]:
pa_classifier=PassiveAggressiveClassifier(max_iter=50)
pa_classifier.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
y_pred=pa_classifier.predict(X_test)

In [ ]:
y_pred=pa_classifier.predict(X_test)

report = classification_report( y_test, y_pred )
print(report)
test, preds = convert_labels(y_test, y_pred)

# Added to remove the error of ValueError: Unknown label type
# test = test.astype('int')
# preds = preds.astype('int')
# print('Log Loss:', metrics.log_loss(test, preds))

              precision    recall  f1-score   support

          SW       0.88      0.88      0.88      3144
          WO       0.85      0.52      0.65      2786
          WV       0.68      0.91      0.78      3150

    accuracy                           0.78      9080
   macro avg       0.80      0.77      0.77      9080
weighted avg       0.80      0.78      0.77      9080

